In [1]:
#!pip install grequests

In [2]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="grequests")
os.environ['GEVENT_SUPPORT'] = 'True'
#import grequests
import requests
from bs4 import BeautifulSoup
import json
import concurrent.futures
import urllib3
import time

In [3]:
####### INUTIL #############
#pd.read_csv("annotators.csv")

In [4]:
#pd.read_excel("annotations.xlsx")

In [ ]:
df = pd.read_excel("labeled_dataset.xlsx").drop(["Unnamed: 0"], axis = 1)

In [6]:
df.dropna(subset=['news_link'], inplace=True)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df['type'].unique()

In [ ]:
df['topic'].unique()

In [11]:
def get_news_batch(urls):
    results = []
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(get_news, i, url) for i, url in enumerate(urls)]
        
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            results.append(result)
    
    return dict(results)

In [12]:
def get_news(index, url):
    try:
        http = urllib3.PoolManager()
        response = http.request("GET", url)
    except:
        #time.sleep(1)
        return (index, ("ERRO - 1"))
        #return get_news(index, url)

    if response.status != 200:
      # Trata a resposta de erro
      if response.status == 429:
        time.sleep(3)
        return get_news(index, url)
      
      soup = BeautifulSoup(response.read(), "html.parser")
      error_message = soup.get_text()
      return (index, (f"ERRO - request: {response.status}"))
    
    soup = BeautifulSoup(response.read(), "html.parser")
    paragrafos = soup.find_all("p")
    
    texto_paragrafos = []
    
    try:
        for paragrafo in paragrafos:
            texto_paragrafos.append(paragrafo.get_text())
    except:
        texto_paragrafos.append("ERRO - 2")

    if len(texto_paragrafos) == 0 :
      soup = BeautifulSoup(response.data, "html.parser")
      paragrafos = soup.find_all("p")
      
      texto_paragrafos = []
      
      try:
          for paragrafo in paragrafos:
              texto_paragrafos.append(paragrafo.get_text())
      except:
          texto_paragrafos.append("ERRO - 3")

    
    return (index, tuple(texto_paragrafos))


In [ ]:
urls = df["news_link"].tolist()
dict_news_text = get_news_batch(urls)

In [ ]:
len(urls)

In [ ]:
len(dict_news_text.keys())

In [16]:
dict_news_series = pd.Series(dict_news_text)

In [17]:
df['full_article'] = dict_news_series

Problematicos:

In [18]:
#zerados = df.loc[df['full_article'].apply(len) == 0]


In [19]:
errados_1 = df.loc[df['full_article'].apply(lambda x: 'ERRO - 1' in str(x))]

In [ ]:
errados_1.shape

In [ ]:
errados_1

In [34]:
errados_2 = df.loc[df['full_article'].apply(lambda x: 'ERRO - 2' in str(x))]

In [ ]:
errados_2.shape

In [36]:
errados_request = df.loc[df['full_article'].apply(lambda x: 'ERRO - request' in str(x))]

In [ ]:
errados_request.shape

In [ ]:
errados_request[['news_link', 'full_article']]

In [ ]:
df_exploded = errados_request.explode("full_article")
valores_unicos = df_exploded["full_article"].unique()
del df_exploded
valores_unicos

In [48]:
merged = df.merge(errados_request, how='left', indicator=True)
filtered_df = merged[merged['_merge'] == 'left_only'].drop(columns='_merge')

filtered_df = filtered_df.merge(errados_1, how= 'left', indicator = True)
filtered_df = filtered_df[filtered_df['_merge'] == 'left_only'].drop(columns='_merge')

In [62]:
filtered_df.dropna(subset = ['full_article'], inplace = True)

In [67]:
filtered_df.to_csv("final_dataset.csv", index = False)